In [ ]:
import cv2
import numpy as np

# --- 1. Load Images and Define Parameters ---
# NOTE: Replace these placeholder images with your actual loaded image data
# full_chip_image = cv2.imread('1.jpg', cv2.IMREAD_GRAYSCALE)
# template_image = cv2.imread('A1_cropped_template.png', cv2.IMREAD_GRAYSCALE)

# For a functional example, assume these are loaded successfully
# We need to simulate the loading/cropping step for the template (A1.png is a crop of 1.jpg)
# Since I cannot perform the crop for you, I'll use placeholders:
# The main chip image '1.jpg' is 3200x3200 (based on typical microfluidic scans)
# The template is a smaller image containing feature '12'

# --- Placeholder Data (You must replace with actual loaded images) ---
try:
    # Attempt to load the images (will fail in this environment, but conceptually necessary)
    full_chip_image = cv2.imread(r"D:\Test1\1024_13_G.png", cv2.IMREAD_GRAYSCALE)
    # The actual template is a crop around the '12' feature
    # A simple estimation based on the image: (let's assume a 400x400 crop around '12')
    # Y-range: 1616 - 800 to 1616 - 400  => 816 to 1216 (approx)
    # X-range: 1603 - 400 to 1603 + 0    => 1203 to 1603 (approx)
    # We will conceptually use a loaded template:
    template_image = cv2.imread(r"D:\Test1\8temp.png", cv2.IMREAD_GRAYSCALE) # Assuming A1.png is the template
    
    # Check if images are loaded successfully
    if full_chip_image is None or template_image is None:
        print("Error: Could not load images. Please ensure file paths are correct.")
        # Create dummy images for demonstration if loading fails
        raise FileNotFoundError
        
except FileNotFoundError:
    print("Using dummy data for demonstration purposes as real loading failed.")
    full_chip_image = np.zeros((3200, 3200), dtype=np.uint8) + 128
    template_image = np.zeros((400, 400), dtype=np.uint8) + 150
    

# Chip Center (from your input)
chip_center_x, chip_center_y = 1616,1607

# Template properties
tw, th = template_image.shape[::-1] # Template width and height (inverted for numpy/cv2)
template_center = (tw // 2, th // 2)

# Parameters for the search
# Given 12 features, a 30-degree step is expected. 
# We use a 1-degree step for high precision to find the absolute best match.
# Search range is 0 to 360 degrees.
angle_step = 1 
angles = np.arange(0, 360, angle_step)

# Variables to store the best result
best_match_value = -1.0 # Correlation score is between -1.0 and 1.0
best_location = (0, 0)
best_angle = 0

# --- 2. Define Rotation Helper Function ---
def rotate_template(img, angle, center):
    """Rotates the template image around its center."""
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), 
                             flags=cv2.INTER_LINEAR, 
                             borderMode=cv2.BORDER_CONSTANT,
                             borderValue=(0)) # Use black border
    return rotated

# --- 3. Iterative Rotation and Matching ---
print(f"Starting rotational search across {len(angles)} angles...")

for angle in angles:
    # a. Rotate the template
    rotated_template = rotate_template(template_image, angle, template_center)
    
    # b. Perform Template Matching (Normalized Cross-Correlation)
    # TM_CCOEFF_NORMED is the standard for robust, normalized matching
    result_map = cv2.matchTemplate(full_chip_image, rotated_template, cv2.TM_CCOEFF_NORMED)
    
    # c. Find the best match value and location for this specific rotation
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result_map)
    
    # d. Update the global best result
    if max_val > best_match_value:
        best_match_value = max_val
        best_location = max_loc
        best_angle = angle

    # Optional: Print progress every 30 degrees
    # if angle % 30 == 0:
    #    print(f"Angle {angle}°: Max Value = {max_val:.4f}")


# --- 4. Final Output ---
print("\n--- Matching Complete ---")
print(f"**Template size:** {tw}x{th} pixels")
print(f"**Search Center:** ({chip_center_x}, {chip_center_y})")

print("\n## 🎉 Highest Match Found")
print(f"**Highest Feature Match Value (Correlation Score):** **{best_match_value:.4f}**")
print(f"**Best Match Location (Top-Left Corner):** {best_location}")
print(f"**Optimal Rotation Angle:** {best_angle}°")

# Calculate the center of the best-matched feature
center_of_match = (best_location[0] + tw // 2, best_location[1] + th // 2)
print(f"**Center of Matched Feature:** {center_of_match}")

Starting rotational search across 360 angles...

--- Matching Complete ---
**Template size:** 186x218 pixels
**Search Center:** (1616, 1607)

## 🎉 Highest Match Found
**Highest Feature Match Value (Correlation Score):** **1.0000**
**Best Match Location (Top-Left Corner):** (34, 1672)
**Optimal Rotation Angle:** 0°
**Center of Matched Feature:** (127, 1781)


In [ ]:
import cv2
import numpy as np

# --- Re-use necessary variables from previous run ---
# Assuming 'full_chip_image', 'template_image', 'best_location',
# 'tw', 'th', 'best_angle', and 'template_center' are defined from the previous script run.

# For demonstration, if you're running this in a new session, you'd need to re-load:
try:
    full_chip_image_color = cv2.imread(r"D:\Test1\1024_13_G.png",cv2.IMREAD_GRAYSCALE) # Load in color for visualization
    template_image = cv2.imread(r"D:\Test1\8temp.png", cv2.IMREAD_GRAYSCALE)
    if full_chip_image_color is None or template_image is None:
        raise FileNotFoundError
except FileNotFoundError:
    print("Using dummy data for visualization. Please ensure images are loaded correctly.")
    full_chip_image_color = np.zeros((3200, 3200, 3), dtype=np.uint8) + 128
    template_image = np.zeros((228, 234), dtype=np.uint8) + 150 # Match the template size from output

# Dummy values for best_location and best_angle if not already set (based on your output)
if 'best_location' not in locals():
    best_location = (1020, 96)
if 'best_angle' not in locals():
    best_angle = 0
if 'tw' not in locals() or 'th' not in locals():
    th, tw = template_image.shape[:2] # height, width
if 'template_center' not in locals():
    template_center = (tw // 2, th // 2)

# Ensure full_chip_image_color is indeed a color image for drawing
if len(full_chip_image_color.shape) == 2: # If it was loaded as grayscale
    full_chip_image_color = cv2.cvtColor(full_chip_image_color, cv2.COLOR_GRAY2BGR)

# --- 1. Prepare the rotated template for drawing ---
# We need the rotated template's dimensions to draw the correct bounding box.
# If the optimal angle is 0, this is just the original template.
def rotate_template_for_drawing(img, angle, center):
    """Rotates the template image around its center."""
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]),
                             flags=cv2.INTER_LINEAR,
                             borderMode=cv2.BORDER_CONSTANT,
                             borderValue=(0)) # Use black border
    return rotated

rotated_template_for_vis = rotate_template_for_drawing(template_image, best_angle, template_center)
# Get actual dimensions of the rotated template (can change if angle is not 0 due to padding)
rot_h, rot_w = rotated_template_for_vis.shape[:2]


# --- 2. Draw the rectangle on the main image ---
# Define the top-left and bottom-right corners of the bounding box
top_left = best_location
bottom_right = (top_left[0] + rot_w, top_left[1] + rot_h)

# Draw a rectangle on the image
# Parameters: image, top-left corner, bottom-right corner, color (BGR), thickness
cv2.rectangle(full_chip_image_color, top_left, bottom_right, (0, 255, 0), 5) # Green rectangle, 5 pixels thick

# Optionally, draw a circle at the center of the matched feature
center_of_match = (best_location[0] + tw // 2, best_location[1] + th // 2)
cv2.circle(full_chip_image_color, center_of_match, 10, (0, 0, 255), -1) # Red circle, filled

# --- 3. Display the result ---
# It's good practice to resize for display if the image is very large
display_image = cv2.resize(full_chip_image_color, (full_chip_image_color.shape[1] // 2, full_chip_image_color.shape[0] // 2)) # Resize to half for viewing

cv2.imshow('Matched Feature on Microfluidic Chip', display_image)
cv2.waitKey(0) # Wait indefinitely until a key is pressed
cv2.destroyAllWindows()

# --- 4. Save the result ---
# You might want to save the annotated image
output_filename = 'matched_feature_annotated.jpg'
cv2.imwrite(output_filename, full_chip_image_color)
print(f"Annotated image saved as '{output_filename}'")

Using dummy data for visualization. Please ensure images are loaded correctly.
